In [5]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA

#BAM
from genai.extensions.langchain import LangChainInterface
from genai.schemas import ModelType, GenerateParams
from genai.model import Credentials
import os
from dotenv import load_dotenv

# make sure you have a .env file under genai root with
GENAI_KEY=""
# GENAI_API=<genai-api-endpoint>
load_dotenv()

True

In [2]:
# Document Loader
from langchain.document_loaders import TextLoader, WebBaseLoader
loader = WebBaseLoader("https://urldefense.com/v3/__https://pitchbook.com/profiles/limited-partner/")
student_doc = loader.load()

# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(student_doc)

# vector store
from langchain.vectorstores import FAISS

embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-large",
            model_kwargs={"device": "cpu"}
        )

docs_db = FAISS.from_documents(docs, embeddings)

find_relavance_query = "First relavent documents for Bill Gates"
find_relavance_query = "describe Bill & Melinda Gates Foundation from the document"

Created a chunk of size 2658, which is longer than the specified 1000


load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
# find_relavance_query = "First relavent documents for Bill Gates"
find_relavance_query = "describe Bill & Melinda Gates Foundation from the document"
relavent_docs = docs_db.similarity_search(find_relavance_query, k=3)
relavent_docs

[Document(page_content='Bill & Melinda Gates Foundation Profile: Commitments & Mandates | PitchBook', metadata={'source': 'https://urldefense.com/v3/__https://pitchbook.com/profiles/limited-partner/11551-42*commitments__;Iw!!HcVu0Bn_hI1r_w!xjDlKihYm_JZqmAlFXKSDMiWy3iLQfoWxlZK-3L7D2x_wJ2A5UwC-9yW8GGHa6OoBwosXmel4h8uWqWa03LhYyiwVUZn2r8$ ', 'title': 'Bill & Melinda Gates Foundation Profile: Commitments & Mandates | PitchBook', 'description': 'Information on commitments, mandates, investment preferences, team and board members for Bill & Melinda Gates Foundation. Use the PitchBook Platform to explore the full profile.', 'language': 'en'}),
 Document(page_content='Foundation\n\n\n\n\n\nFounded\n\n2000\n\n\n\n\n\n\n\n\nProfessionals\n\n7\n\n\n\n\n\n\n\n\nAUM\n\n0000\n\n\n\n\n\nCommitments\n\n25\n\n\n\n\n\n\n\n\nBill & Melinda Gates Foundation General Information\nDescription\nBill & Melinda Gates Foundation is an independent non-profit foundation based in Seattle, Washington. Established in 

In [9]:
# choose a LLM model

model_llm = LangChainInterface(
        model=ModelType.FLAN_T5_11B,
        credentials=Credentials(os.environ.get("GENAI_KEY")),
        params=GenerateParams(
            decoding_method="greedy",
            max_new_tokens=300,
            min_new_tokens=15,
            repetition_penalty=2,
        ).dict()
    )

In [13]:
# no context response from LLM
model_llm.generate(['describe Bill & Melinda Gates Foundation'])

LLMResult(generations=[[Generation(text='Bill & Melinda Gates Foundation is an American private foundation that supports global health and development.', generation_info=None)]], llm_output=None)

In [ ]:
https://urldefense.com/v3/__http://model_llm.ru__;!!HcVu0Bn_hI1r_w!xjDlKihYm_JZqmAlFXKSDMiWy3iLQfoWxlZK-3L7D2x_wJ2A5UwC-9yW8GGHa6OoBwosXmel4h8uWqWa03LhYyiww7FRimA$ 

In [14]:
## Langchain QA
chain = load_qa_chain(model_llm, chain_type='stuff', verbose=False)
prompt = "Extract the description from the content"

## Run the Chain
chain.run(input_documents=relavent_docs, question=query)

'Bill & Melinda Gates Foundation is an independent non-profit foundation based in Seattle, Washington. Established in 2000, the foundation works to reduce inequities around the world. It focuses on improving health and alleviating extreme poverty. The assets of the foundation are managed by the board of trustees.'

In [10]:
model_llm.generate(['describe Bill & Melinda Gates Foundation'])

LLMResult(generations=[[Generation(text='Bill & Melinda Gates Foundation is an American private foundation that supports global health and development.', generation_info=None)]], llm_output=None)